In [ ]:
!pip install kaggle #Instalando a biblioteca do Kaggle para posteriormente baixar o dataset que irá ser utilizado

In [ ]:
#Função para importar o Token (chave de acesso) do Kaggle para baixar diretamente do colab

from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rishitchs/final-flowers-course-project-dataset #Instalação do Dataset

 96% 410M/426M [00:02<00:00, 140MB/s]
100% 426M/426M [00:02<00:00, 165MB/s]


In [ ]:
#Importando bibliotecas necessárias

from keras.preprocessing import image
from zipfile import ZipFile
import pandas as pd 
from pathlib import Path
import os
import timeit
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from keras.layers import Conv2D,Dense,Dropout,Input,Flatten,MaxPooling2D
from keras.models import Model
from keras.utils import plot_model,to_categorical
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.optimizers import *
from keras.datasets import cifar10
from keras.layers import Dropout, BatchNormalization
from keras.models import Model
from keras.utils import to_categorical, plot_model
from collections import Counter
from keras.backend import clear_session
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import keras.layers as kl
import tensorflow as tf
from sklearn.metrics import classification_report
%matplotlib inline

In [ ]:
# Adicionando o arquivo zip a uma variável
arquivo = "final-flowers-course-project-dataset.zip"
  
# Descompactando o arquivo zip
with ZipFile(arquivo, 'r') as zip:
    zip.printdir()
  
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

In [ ]:
#Importando as funções para o data argumentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#Armazenando o Path de cada label para usar na função
rose = Path('/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/rose')

daisy = Path('/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/daisy')

dandelion = Path('/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/dandelion')

sunflower = Path('/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/sunflower')

tulip = Path('/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/tulip')

In [ ]:
#Definindo as modificações que serão aplicadas nas imagens
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

#Criando uma função para criar novas imagens a parir da função do data argumentation e já armazenando nas devidas pastas
def flower_aug(flower, name, n):
  for imagem in flower.glob('*'): 
    img = load_img(imagem)  
    x = img_to_array(img)  
    x = x.reshape((1,) + x.shape)  

  i = 0
  for batch in datagen.flow(x, batch_size=1,
                            save_to_dir= flower, save_prefix=name, save_format='jpg'):
      i += 1
      if i > n:
          break


In [ ]:
flores = []

In [ ]:
#Utilizando a função para cada label 
flower_aug(sunflower, 'sunflower_aug', 406 )

flower_aug(tulip, 'tulip_aug', 166)

flower_aug(dandelion, 'dandelion_aug', 48)

flower_aug(rose, 'rose_aug', 356)

flower_aug(daisy, 'daisy_aug', 331)

In [ ]:
#Após a criação de novas imagens, utilizamos o glob para percorrer e pegar os arquivos de cada pasta e armazenando em listas em formato de matriz
p = Path('/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/')
dirs = p.glob('*')
image_data = []
labels = []
image_paths = []
labels_dict = {'dandelion':0,'daisy':1,'sunflower':2,'tulip':3,'rose':4}

for fold in dirs:
  label =str(fold).split('/')[-1]

  cnt = 0
  print(fold)
  for img_path in fold.glob("*.jpg"):
    img = image.load_img(img_path, target_size=(64,64))
    img_array = image.img_to_array(img)
    image_data.append(img_array)

    labels.append(labels_dict[label])

    cnt +=1
  print(cnt)

/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/sunflower
1173
/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/dandelion
1516
/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/tulip
1420
/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/rose
1215
/content/Flowers_Dataset_Revised/Flowers_Dataset_Revised/newFlowers/daisy
1235


In [ ]:
print(len(image_data),len(labels))

6559 6559


In [ ]:
#Definindo nosso X e nosso y
X = np.array(image_data)
y = np.array(labels)

In [ ]:
#Divisão dos dados 
data = train_test_split(X,y,test_size=0.2, shuffle=True, stratify=y)
X_train, X_test, y_train, y_test = data

In [ ]:
#Plotando as imagens sem reescalar os dados

plt.figure(figsize=(10,10))
for i in range(0,9):
  plt.subplot(330 + 1 + i)
  # plot raw pixel data
  plt.imshow(X_train[i])
# show the figure
plt.show()

In [ ]:
#Plotando as imagens reescalando os dados por 255

plt.figure(figsize=(10,10))
for i in range(0,9):
  plt.subplot(330 + 1 + i)
  # plot raw pixel data
  plt.imshow(X_train[i]/255.0)
# show the figure
plt.show()

In [ ]:
#Plotando exemplos da Label Rose

for flor in flores:
  rose = flores[0]
image_list = []
for filename in rose.glob('*'): 
    im=Image.open(filename)
    image_list.append(im)

plt.figure(figsize=(10,10))
for i in range(9):
  plt.subplot(330 + 1 + i)
  plt.imshow(image_list[i])
plt.show()

In [ ]:
#Plotando exemplos da Label Dandelion

for flor in flores:
  dandelion = flores[1]
image_list = []
for filename in dandelion.glob('*'): 
    im=Image.open(filename)
    image_list.append(im)

plt.figure(figsize=(10,10))
for i in range(9):
  plt.subplot(330 + 1 + i)
  plt.imshow(image_list[i])
plt.show()

In [ ]:
#Plotando exemplos da Label Sunflower

for flor in flores:
   sunflower= flores[2]
image_list = []
for filename in sunflower.glob('*'): 
    im=Image.open(filename)
    image_list.append(im)

plt.figure(figsize=(10,10))
for i in range(9):
  plt.subplot(330 + 1 + i)
  plt.imshow(image_list[i])
plt.show()

In [ ]:
#Plotando exemplos da Label Tulip

for flor in flores:
   tulip = flores[3]

image_list = []
for img in tulip.glob('*'): 
    im=Image.open(img)
    image_list.append(im)

plt.figure(figsize=(10,10))
for i in range(9):
  plt.subplot(330 + 1 + i)
  plt.imshow(image_list[i])
plt.show()

In [ ]:
#Plotando exemplos da Label Daisy

for flor in flores:
   daisy = flores[4]
image_list = []
for img in daisy.glob('*'): 
    im=Image.open(img)
    image_list.append(im)

plt.figure(figsize=(10,10))
for i in range(9):
  plt.subplot(330 + 1 + i)
  plt.imshow(image_list[i])
plt.show()

In [ ]:
#Verificando a quantidade de dados para cada label

plt.figure(figsize=(15,5))
plt.bar(Counter(y_test).keys(), Counter(y_test).values(), align='center', alpha=0.5)
plt.ylabel('Instancias')
plt.xlabel('Classe')
plt.title('Instancias por classe')
plt.show()

In [ ]:
#Tratando os dados para o treino
#OBS: Comentar esse trecho se for rodar o K-Fold

# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

# X_train /= 255
# X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

5247 train samples
1312 test samples


In [ ]:
#Verificando a quantidade de classes

num_classes = len(Counter(y_test))
num_classes

5

In [ ]:
#Fazendo o One Hot Encoder para as variáveis y
#OBS: Comentar esse trecho se for rodar o K-Fold

# y_train = to_categorical(y_train, num_classes)
# y_test = to_categorical(y_test, num_classes)

In [ ]:
#Função do modelo

def get_model(nb_classes = 5, input_shape = (64,64,3)):

  model = Sequential()
  model.add(kl.Input(input_shape))
  model.add(kl.Conv2D(64, (3,3), padding = 'same',activation='relu'))
  model.add(kl.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(kl.Conv2D(96, (3,3),padding = 'same',activation='relu'))
  model.add(kl.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(kl.Dropout(0.2))
  model.add(BatchNormalization())
  model.add(kl.Conv2D(512, (3,3),padding = 'same',activation='relu'))
  model.add(kl.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(kl.Dropout(0.3))
  model.add(BatchNormalization())
  model.add(kl.Conv2D(64, (3,3),padding = 'same',activation='relu'))
  model.add(kl.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(kl.Dropout(0.25))
  model.add(BatchNormalization())
  model.add(kl.Conv2D(32, (3,3),padding = 'same',activation='relu'))
  model.add(kl.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(kl.Conv2D(15, (2,2)))
  model.add(kl.Flatten())
  model.add(kl.Dense(1024, activation='relu'))
  model.add(kl.Dropout(0.3))
  model.add(kl.Dense(nb_classes, activation='softmax'))


  # Apresentando a estrutura do modelo
  model.summary()

  image = tf.expand_dims(X_train[1], 0)

  # Compilando o modelo
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(), 
                metrics=['accuracy'])

  # Retornando o modelo
  return model

In [ ]:
# Visualizando o modelo

model = get_model(num_classes, X_test.shape[1:])
plot_model(model, show_dtype=True, show_shapes=True, show_layer_names=True)

In [ ]:
#Importando a biblioteca para uma visualização do modelo por outro ângulo
!pip install visualkeras
import visualkeras

visualkeras.layered_view(model)

In [ ]:
# Quantidade de épocas
epochs = 100
# Batch size para treinamento
bs = 30

Treinando o modelo, e armazenando o resultado do treinamento em uma variável
hist = model.fit(X_train, y_train,
                 epochs = epochs,
                 batch_size = bs,
                 validation_split=0.20,
                 verbose=1)

# Computando o tempo para treinar (fim)
stop = timeit.default_timer()

In [ ]:
y_pred = model.predict(X_test)
n_predict = np.argmax(y_pred,axis = 1)
n_y_test = np.argmax(y_test, axis = 0)

In [ ]:
#Criando uma matriz de confusão

from sklearn.metrics import confusion_matrix
data = {'y_test':n_y_test,
        'pred': n_predict}

df = pd.DataFrame(data, columns=['y_test','pred'])

matriz = pd.crosstab(df['y_test'], df['pred'], rownames=['Realidade'], colnames=['Predições'])
print(matriz)

In [ ]:
# Criando o classification report para a comparação de diversos métricas

target_names = ['Daisy', 'Dandelion', 'Rose','Sunflower','Tulip']
print(classification_report(n_y_test, n_predict, target_names=target_names))

In [ ]:
# Evaluate do modelo
scores = model.evaluate(X_test, y_test, verbose=0)

# Gráfico de loss
plt.figure(figsize=(10,8))
plt.title(f"Gráfico de Loss - {scores[0]}")
plt.xlabel("Épocas")
plt.ylabel("Valor")
plt.plot(hist.history['loss'], label="loss")
plt.plot(hist.history['val_loss'], label="val_loss")
plt.legend()
plt.show()

# Gráfico de acurácia
plt.figure(figsize=(10,8))
plt.title(f"Gráfico de Acurácia - {scores[1]}")
plt.xlabel("Épocas")
plt.ylabel("Valor")
plt.plot(hist.history['accuracy'], label="accuracy")
plt.plot(hist.history['val_accuracy'], label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
# Chamamos a função StratifiedKFold para realizar um particionamento dos dados durante o treinamento
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)

hist_scores = []
#Separação dos dados do K-flod em treino e teste
for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):

    # print('Fold {}'.format(i))
    # print('\tÍndices para conjunto de treinamento:')
    # print('{}'.format(train_index))
    # print('\tÍndices para conjunto de teste:')
    # print('{}'.format(test_index))
    # print()
    
#Separação dos dados em treino e teste , e também em x e y
    KX_train = X[train_index]
    KX_test = X[test_index]
    Ky_train = y[train_index]
    Ky_test = y[test_index]
#Verificar o número de classes presentes
    num_classes = 5

    KX_train = KX_train.astype('float32')
    KX_test = KX_test.astype('float32')
#Redimensonamento dos dados 
    KX_train /= 255
    KX_test /= 255
#Passamos os dados para o to_categorical 
    Ky_train = to_categorical(Ky_train, 5)
    Ky_test = to_categorical(Ky_test, 5)

    # print('Fold {}'.format(i))
    # print('\tDados para conjunto de treinamento:')
    # print('{}'.format(X_train))
    # print('{}'.format(y_train))
    # print('\tDados para conjunto de teste:')
    # print('{}'.format(X_test))
    # print('{}'.format(y_test))
    # print()



    model = get_model(num_classes, KX_test.shape[1:])
#treinamento da rede neural com o k-fold
    hist = model.fit(KX_train, Ky_train,
                 epochs = 30,
                 batch_size = 300,
                 validation_split= 0.2,
                 verbose=1)
    #Avaliação do modelo 
    scores = model.evaluate(KX_test, Ky_test, verbose=0)
#Armazenamento do desempenho em uma lista
    hist_scores.append(scores)

In [ ]:
#K-Fold com dados
df = hist_scores

tabela = pd.DataFrame(df, columns=['Loss','Accuracy'])

tabela